In [ ]:
!pip install psycopg2

### Connect with the DB

In [ ]:
import psycopg2

conn = psycopg2.connect(
    database = "biowhere-gazetteer",
    user = "postgres",
    host = "127.0.0.1",
    password = "<you-password>",
    port = 5432)

cur = conn.cursor()

### Create DB Schema

In [ ]:
# CREATE TABLES

cur.execute("""

CREATE EXTENSION postgis;

create type TimePeriod AS (
    isCurrent boolean,
    startDate date,
    startDatePrecision timestamp,
    endDate date,
    endDatePrecision timestamp,
    dateSource text
);

create type Pronunciation AS (
    pronunciationUsage text,
    pronunciationText text,
    pronunciationAudio text
);

create type SpatialAccuracy AS (
    value text,
    scheme text
);

CREATE TABLE LocalityDescription (
    id serial not null,
    localityDescription text,
    lastUpdateDate date,
    lastUpdateUser text,
    primary key(id)
);

create table Feature (
    id serial not null,
    featureDescription text,
    inferredFlag boolean,
    lastUpdateDate date,
    lastUpdateUser text,
    primary key(id)
);

create table SpatialGeometryRepresentation (
    id serial not null,
    lastUpdateDate date,
    lastUpdateUser text,
    timePeriod TimePeriod,
    spatialAccuracy SpatialAccuracy,
    primary key(id),

    feature_id bigint,
    constraint ft foreign key(feature_id)
        references Feature(id) on delete restrict,
    localityDescription_id bigint,
    constraint ld foreign key(localityDescription_id)
        references LocalityDescription(id) on delete restrict
);

create table SpatialGeometryRepresentation_point (
    id serial not null,
    geodeticReferenceSystem text not null,
    geometry geometry not null,
    lastUpdateDate date,
    lastUpdateUser text,
    primary key(id),

    spatialGeometryRepresentation_id bigint,
    constraint sgr foreign key(spatialGeometryRepresentation_id)
        references SpatialGeometryRepresentation(id) on delete restrict
);

create table SpatialGeometryRepresentation_line (
    id serial not null,
    geodeticReferenceSystem text not null,
    geometry geometry not null,
    lastUpdateDate date,
    lastUpdateUser text,
    primary key(id),

    spatialGeometryRepresentation_id bigint,
    constraint sgr foreign key(spatialGeometryRepresentation_id)
        references SpatialGeometryRepresentation(id) on delete restrict
);

create table SpatialGeometryRepresentation_polygon (
    id serial not null,
    geodeticReferenceSystem text not null,
    geometry geometry not null,
    lastUpdateDate date,
    lastUpdateUser text,
    primary key(id),

    spatialGeometryRepresentation_id bigint,
    constraint sgr foreign key(spatialGeometryRepresentation_id)
        references SpatialGeometryRepresentation(id) on delete restrict
);

create type classificationSchemeEnum as enum();

create table FeatureType (
    id serial not null,
    classificationScheme text,
    featureClass text,
    lastUpdateDate date,
    lastUpdateUser text,
    primary key(id),

    feature_id bigint,
    constraint ft foreign key(feature_id)
        references Feature(id) on delete restrict
);

create table FeatureName (
    id serial not null,
    featureName text not null,
    language text,
    nameUsage text,
    offensiveFlag boolean,
    validatedFlag boolean,
    pronunciation Pronunciation,
    lastUpdateDate date,
    lastUpdateUser text,
    primary key(id),

    feature_id bigint,
    constraint ft foreign key(feature_id)
        references Feature(id) on delete restrict
);

create type featureRealtionEnum as enum();

create table FeatureRelation (
    id serial not null,
    featureRelationType text,
    lastUpdateDate date,
    lastUpdateUser text,
    primary key(id)
);

create table Feature_FeatureRelation (
    id serial not null,
    primary key(id),
    lastUpdateDate date,
    lastUpdateUser text,

    feature_id bigint,
    constraint ft foreign key(feature_id)
        references Feature(id) on delete restrict,

    featureRelation_id bigint,
    constraint fr foreign key(featureRelation_id)
        references FeatureRelation(id) on delete restrict
);


create type externalSourceIdEnum as enum();


create table Source (
    id serial not null,
    externalSourceId int,
    source text,
    preferredFlag boolean,
    lastUpdateDate date,
    lastUpdateUser text,
    primary key(id),

    spatialGeometryRepresentation_id bigint,
    constraint sgr foreign key(spatialGeometryRepresentation_id)
        references SpatialGeometryRepresentation(id) on delete restrict,

    featureType_id bigint,
    constraint fty foreign key(featureType_id)
        references FeatureType(id) on delete restrict,

    featureName_id bigint,
    constraint fn foreign key(featureName_id)
        references FeatureName(id) on delete restrict,

    localityDescription_id bigint,
    constraint ld foreign key(localityDescription_id)
        references LocalityDescription(id) on delete restrict
);




create type featureNameRelationTypeEnum as enum();

create table FeatureNameRelation (
    id serial not null,
    featureNameRelationType text,
    lastUpdateDate date,
    lastUpdateUser text,
    primary key(id)
);

create table FeatureName_FeatureNameRelation (
    id serial not null,
    lastUpdateDate date,
    lastUpdateUser text,
    primary key(id),

    featureName_id bigint,
    constraint fn foreign key(featureName_id)
        references FeatureName(id) on delete restrict,

    featureNameRelation_id bigint,
    constraint fnr foreign key(featureNameRelation_id)
        references FeatureNameRelation(id) on delete restrict
);

create table Abbreviation (
    id serial not null,
    abbreviation text,
    lastUpdateDate date,
    lastUpdateUser text,
    primary key(id)
);

create table FeatureName_Abbreviation (
    id serial not null,
    lastUpdateDate date,
    lastUpdateUser text,
    primary key(id),

    featureName_id bigint,
    constraint fn foreign key(featureName_id)
        references FeatureName(id) on delete restrict,


    abbreviation_id bigint,
    constraint ab foreign key(abbreviation_id)
        references Abbreviation(id) on delete restrict
);

create table Whakapapa (
    id serial not null,
    whakapapa text,
    whakapapaUsage text,
    doi text,
    citation text,
    url text,
    lastUpdateDate date,
    lastUpdateUser text,
    primary key(id)
);

create table FeatureName_Whakapapa (
    id serial not null,
    lastUpdateDate date,
    lastUpdateUser text,
    primary key(id),

    featureName_id bigint,
    constraint fn foreign key(featureName_id)
        references FeatureName(id) on delete restrict,

    whakapapa_id bigint,
    constraint wp foreign key(whakapapa_id)
        references Whakapapa(id) on delete restrict
);

create table AlternativeSpelling (
    id serial not null,
    alternativeSpelling text,
    spellingUsage text,
    lastUpdateDate date,
    lastUpdateUser text,
    primary key(id),

    featureName_id bigint,
    constraint fn foreign key(featureName_id)
        references FeatureName(id) on delete restrict
);

create table IntegrationSource (
    id serial not null,
    integratedRecordId int,
    lastUpdateDate date,
    lastUpdateUser text,
    primary key(id),

    source_id bigint,
    constraint sr foreign key(source_id)
    references Source(id) on delete restrict
);


create table Feature_LocalityDescription (
    id serial not null,
    lastUpdateDate date,
    lastUpdateUser text,
    primary key(id),

    feature_id bigint,
    constraint ft foreign key(feature_id)
        references Feature(id) on delete restrict,

    localityDescription_id bigint,
    constraint ld foreign key(localityDescription_id)
        references LocalityDescription(id) on delete restrict
);


""")

conn.commit()
cur.close()
conn.close()